# Lithium-ion battery cell State of Health model


## Introduction

When using a battery based on lithium-ion battery cells, it's important to monitor how these energy cells deprecate over time. This can be done by tracking the value of different parameters such as State of Health (SoH). This is the ratio of the discharge capacity of the cell and it's rated capacity. The formula for it is:
$$SoH = \frac{C_{discharge}}{C_{rated}}$$
The rated capacity $C_{rated}$ for a lion cell is constant and the discharge capacity $C_{discharge}$ changes with battery usage, so by estimating the discharge capacity we can get SoH, which will tell us about battery usage.
  
The aim of this project is to estimate the SoH of a lion battery cell based on discharge data features. Through data exploration (see notebook titled `discharge_data_visualization.ipynb`) we came to a conclusion that a model which predicts battery SoH based on the discharge current, voltage and/or temperature timeseries may be a good fit to perform this task.  

The models we propose here focus on features extracted from timeseries of current, voltage and temperature measurements.

Conveniently, the NASA battery dataset is fitting for this task, as it contains a huge number of measurements of discharge current, voltage, temperature as timeseries and, most importantly, the discharge capacity we're interested in.

This means that via careful feature extraction from these timeseries we can test the proposed models.



## Data

The NASA battery dataset ([link - using an already pre-processed version to accelerate data exploration](https://www.kaggle.com/datasets/patrickfleith/nasa-battery-dataset)) provides all required data to begin feature extraction. Those contain battery voltage, current and temperature under constant load of -2[A]. The measurements start with the battery full, at 4.2[V] and go all the way to it's lowest safe discharge voltage of around 2.4-2.7[V], depending on the exact battery.  

Our goal is to prepare a model general enough to include all types of battery cells, which is why we will be experimenting with a number of features. Those are:
1. Starting voltage.
2. End voltage.
3. Time under load (constant current peak to peak time).
4. TODO(treborsky): update the features as we include more of them in the models
5. ...
6. ...

The method and calculation of the features is described in more detail below, next to the implementation.



## DAG

TODO(treborsky): prepare the DAG for our data


## Data Extraction

In [2]:
import pandas as pd
import csv
from tqdm import tqdm

# Picked data
picked_data = {
    'battery_id': ['B0005', 'B0006', 'B0007'],
    'type': ['discharge']
}

metadata_path = 'cleaned_dataset/metadata.csv'
data = {}

# Read metadata.csv and creating data dictionary
with open(metadata_path, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        battery_id = row['battery_id']
        if battery_id in picked_data['battery_id']:
            data[battery_id] = {
                'charge': {},
                'discharge': {},
                'impedance': {}
            }

data_path = 'cleaned_dataset/data/'

cycle_ids = {
    'charge': 0,
    'discharge': 0,
    'impedance': 0
}

# Read and store metadata information for analyzed batteries
metadata = pd.read_csv('cleaned_dataset/metadata.csv')

# Create a progress bar
pbar = tqdm(total=len(metadata))

# Iterate over each row in the metadata dataframe
for index, row in metadata.iterrows():
    if index == 0:
        metadata_headers = row.keys()

    if row['battery_id'] != battery_id:
        cycle_ids = {
            'charge': 0,
            'discharge': 0,
            'impedance': 0
        }
    type = row['type']
    cycle_id = cycle_ids[type]
    battery_id = row['battery_id']

    if type in picked_data['type'] and battery_id in picked_data['battery_id']:
        values_metadata = {}
        # To values append non-empty values of metadata headers
        for header in metadata_headers:
            if row[header] != '':
                values_metadata[header] = row[header]

        filename = row['filename']

        with open(data_path+filename, 'r') as csvfile:
            csvreader = csv.DictReader(csvfile)
            headers = csvreader.fieldnames
            values = {header: [] for header in headers}
            for row in csvreader:
                for header in headers:
                    values[header].append(row[header])
            
            # Concatenate metadata and data values
            values = {**values_metadata, **values}
            data[battery_id][type][cycle_id] = values
            csvfile.close()
        cycle_ids[type] += 1

    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()


100%|██████████| 7565/7565 [00:02<00:00, 2732.66it/s] 


## Data preprocessing

The extraction of features is done in a common way for all datasets for all batteries.

TODO(treborsky): Add feature extraction. Start with the ones already defined, build the first model (ppc, then fit) and go back here to update if more features needed/wanted.

In [3]:
from feature_extraction import features
import numpy as np

# function array string to array float
def array_str_to_float(array_str):
    array_float = []
    for i in range(len(array_str)):
        array_float.append(float(array_str[i]))
    return array_float

# add features to data dictionary
for battery_id in data.keys():
    for type in data[battery_id].keys():
        for cycle_id in data[battery_id][type].keys():
            if type == 'discharge':
                data[battery_id][type][cycle_id]['start_voltage'] = features.get_start_voltage(
                    pd.Index(array_str_to_float(data[battery_id][type][cycle_id]['Voltage_measured'])))
                data[battery_id][type][cycle_id]['stop_voltage'] = features.get_stop_voltage(
                    pd.Index(array_str_to_float(data[battery_id][type][cycle_id]['Voltage_measured'])))
                data[battery_id][type][cycle_id]['current_ptp'] = features.get_current_ptp(
                    pd.Index(array_str_to_float(data[battery_id][type][cycle_id]['Current_measured'])),
                    pd.Index(array_str_to_float(data[battery_id][type][cycle_id]['Time'])))


## Models

### Model 1

The first model uses only the features that are the easiest to extract from existing data. Those are:
1. Start voltage - $x_0$.
2. End voltage - $x_1$.
3. Time under load (constant discharge current peak to peak time) - $x_2$.

Let's say this data forms a vector $X_1 = \begin{bmatrix} x_0 \\ x_1 \\ x_2 \end{bmatrix} $

$$ y \sim Normal(\alpha + \beta\cdot X, \sigma) $$